In [1]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon

In [2]:
p_taipei = pd.read_csv("points_taipei.csv")
p_taipei['index'] = range(0, len(p_taipei))

p_new_taipei = pd.read_csv("points_new_taipei.csv")
p_new_taipei['index'] = range(0, len(p_new_taipei))

def create_geometry_gdf(df):
    geometry = [Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]) for x1, x2, y1, y2 in zip(df['x1'], df['x2'], df['y1'], df['y2'])]
    geometry_gdf = gpd.GeoDataFrame(geometry=geometry)
    geometry_gdf['index'] = range(0, len(df))
    return geometry_gdf

geometry_gdf_t = create_geometry_gdf(p_taipei)
geometry_gdf_nt = create_geometry_gdf(p_new_taipei)

In [3]:
def other_data(file_path):
        df = pd.read_csv(file_path, header=0)
        park = df[df['CATEGORY'].isin(['休閒公園'])]
        industry = df[df['CATEGORY'].isin(['工業產業園區'])]
        bad = df[df['CATEGORY'].isin(['殯葬設施','垃圾場', '焚化廠'])]
        park_100_gdf, park_500_gdf = park_count(park)
        industry_500_gdf, industry_1000_gdf, industry_2000_gdf = industry_count(industry)
        bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf = bad_count(bad)
        return park_100_gdf, park_500_gdf, industry_500_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf
        
def park_count(park):
        # 創建一個包含公園位置的GeoDataFrame
        park_points1 = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(park['POI_X'], park['POI_Y'])])

        park_100 = park_points1.buffer(100)
        park_500 = park_points1.buffer(500)

        park_100_gdf = gpd.GeoDataFrame(geometry=park_100, crs=park_points1.crs)
        park_500_gdf = gpd.GeoDataFrame(geometry=park_500, crs=park_points1.crs)
        return park_100_gdf, park_500_gdf

def industry_count(industry):
        # 創建一個包含工業園區位置的GeoDataFrame
        industry_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(industry['POI_X'], industry['POI_Y'])])

        industry_500 = industry_points.buffer(500)
        industry_1000 = industry_points.buffer(1000)
        industry_2000 = industry_points.buffer(2000)

        industry_500_gdf = gpd.GeoDataFrame(geometry=industry_500, crs=industry_points.crs)
        industry_1000_gdf = gpd.GeoDataFrame(geometry=industry_1000, crs=industry_points.crs)
        industry_2000_gdf = gpd.GeoDataFrame(geometry=industry_2000, crs=industry_points.crs)
        return industry_500_gdf, industry_1000_gdf, industry_2000_gdf
    
def bad_count(bad):
        # 創建一個包含反向效果位置的GeoDataFrame
        bad_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in zip(bad['POI_X'], bad['POI_Y'])])

        bad_100 = bad_points.buffer(100)
        bad_500 = bad_points.buffer(500)
        bad_1000 = bad_points.buffer(1000)
        bad_2000 = bad_points.buffer(2000)

        bad_100_gdf = gpd.GeoDataFrame(geometry=bad_100, crs=bad_points.crs)
        bad_500_gdf = gpd.GeoDataFrame(geometry=bad_500, crs=bad_points.crs)
        bad_1000_gdf = gpd.GeoDataFrame(geometry=bad_1000, crs=bad_points.crs)
        bad_2000_gdf = gpd.GeoDataFrame(geometry=bad_2000, crs=bad_points.crs)
        return bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf

def overlay_and_count(geometry_gdf, other_gdf, prefix):
    intersection = geometry_gdf.overlay(other_gdf, how='intersection')
    count_df = intersection.groupby('index').size().reset_index(name=prefix + '_cnt')
    return count_df

def calculate_other_counts(geometry_gdf, park_100_gdf, park_500_gdf, industry_500_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf):
    park_100_cnt = overlay_and_count(geometry_gdf, park_100_gdf, 'park_100')
    park_500_cnt = overlay_and_count(geometry_gdf, park_500_gdf, 'park_500')
    industry_500_cnt = overlay_and_count(geometry_gdf, industry_500_gdf, 'industry_500')
    industry_1000_cnt = overlay_and_count(geometry_gdf, industry_1000_gdf, 'industry_1000')
    industry_2000_cnt = overlay_and_count(geometry_gdf, industry_2000_gdf, 'industry_2000')
    bad_100_cnt = overlay_and_count(geometry_gdf, bad_100_gdf, 'bad_100')
    bad_500_cnt = overlay_and_count(geometry_gdf, bad_500_gdf, 'bad_500')
    bad_1000_cnt = overlay_and_count(geometry_gdf, bad_1000_gdf, 'bad_1000')
    bad_2000_cnt = overlay_and_count(geometry_gdf, bad_2000_gdf, 'bad_2000')
    return park_100_cnt, park_500_cnt, industry_500_cnt, industry_1000_cnt, industry_2000_cnt, bad_100_cnt, bad_500_cnt, bad_1000_cnt, bad_2000_cnt


def calculate_weighted_score(df, *args):
    # 進行多個 merge 操作
    for arg in args:
        df = df.merge(arg, on='index', how='left')
    
    # 將 NaN 值填充為 0
    df = df.fillna(0)
    
    # 計算加權分數
    df['公園加權分數'] = (
        df['park_100_cnt'] * 0.5 + 
        df['park_500_cnt'] * 0.8
    )
    df['工業區加權分數'] = (
        df['industry_500_cnt'] * 0.2 + 
        df['industry_1000_cnt'] * 0.4 + 
        df['industry_2000_cnt'] * 0.5
    )
    df['反向效果加權分數'] = (
        df['bad_100_cnt'] * 1 + 
        df['bad_500_cnt'] * 0.8 + 
        df['bad_1000_cnt'] * 0.5 + 
        df['bad_2000_cnt'] * 0.3
    )
    
    return df


In [4]:
for i in range(2019,2025):
    park_100_gdf, park_500_gdf, industry_500_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf = other_data(f"data_yaer\{i}_other.csv")
    
    # 呼叫函式
    park_100_cnt_t, park_500_cnt_t, industry_500_cnt_t, industry_1000_cnt_t, industry_2000_cnt_t, bad_100_cnt_t, bad_500_cnt_t, bad_1000_cnt_t, bad_2000_cnt_t = calculate_other_counts(geometry_gdf_t, park_100_gdf, park_500_gdf, industry_500_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf)
    park_100_cnt_nt, park_500_cnt_nt, industry_500_cnt_nt, industry_1000_cnt_nt, industry_2000_cnt_nt, bad_100_cnt_nt, bad_500_cnt_nt, bad_1000_cnt_nt, bad_2000_cnt_nt = calculate_other_counts(geometry_gdf_nt, park_100_gdf, park_500_gdf, industry_500_gdf, industry_1000_gdf, industry_2000_gdf, bad_100_gdf, bad_500_gdf, bad_1000_gdf, bad_2000_gdf)

    # 呼叫函式
    result_t = calculate_weighted_score(p_taipei, park_100_cnt_t, park_500_cnt_t, industry_500_cnt_t, industry_1000_cnt_t, industry_2000_cnt_t, bad_100_cnt_t, bad_500_cnt_t, bad_1000_cnt_t, bad_2000_cnt_t)
    result_nt = calculate_weighted_score(p_new_taipei, park_100_cnt_nt, park_500_cnt_nt, industry_500_cnt_nt, industry_1000_cnt_nt, industry_2000_cnt_nt, bad_100_cnt_nt, bad_500_cnt_nt, bad_1000_cnt_nt, bad_2000_cnt_nt)
    
    # 寫入csv
    result_t.to_csv(f"year_score/{i}_taipei_other.csv", index=False)
    result_nt.to_csv(f"year_score/{i}_new_taipei_other.csv", index=False)
    print(i)

2019
2020
2021
2022
2023
2024
